<a href="https://colab.research.google.com/github/sudhirshahu51/RAG/blob/main/Building_Agentic_RAG_with_llamaIndex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing required libraries**

In [37]:
%%writefile requirements.txt

llama-index==0.10.27
llama-index-llms-text-generation-inference
llama-index-llms-huggingface
llama-index-llms-huggingface
sentence-transformers #for embedding model
llama-index-embeddings-huggingface
llama-index-embeddings-instructor
#llama-index-llms-openai==0.1.15
#llama-index-embeddings-openai==0.1.7

Overwriting requirements.txt


In [38]:
!pip install -r requirements.txt --quiet

In [39]:
# to make async work well with jupyter notebook
import nest_asyncio
nest_asyncio.apply()

**Login for HuggingFace API for LLM**

In [40]:
import os
from google.colab import userdata
key = userdata.get('HUGGING_FACE')
os.environ['HUGGINGFACEHUB_API_TOKEN'] = key

In [41]:
from typing import List, Optional

from llama_index.core import Settings
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [42]:
# This will use the same model, but run remotely on Hugging Face's servers,
# accessed via the Hugging Face Inference API
# Note that using your token will not charge you money,
# the Inference API is free it just has rate limits

remotely_run = HuggingFaceInferenceAPI(model_name="HuggingFaceH4/zephyr-7b-alpha", token=key)

# Or you can skip providing a token, using Hugging Face Inference API anonymously
remotely_run_anon = HuggingFaceInferenceAPI(model_name="HuggingFaceH4/zephyr-7b-alpha")

In [43]:
llm = HuggingFaceInferenceAPI(model_name="HuggingFaceH4/zephyr-7b-alpha", token=key)


In [94]:
completion_response = llm.complete("To infinity, and")
print(completion_response)


 beyond!

The Toy Story franchise has been a beloved part of pop culture for over two decades, and it's not slowing down anytime soon. The latest installment, Toy Story 4, is set to hit theaters this summer, and it's already generating buzz.

The movie follows the adventures of Woody, Buzz, and the gang as they embark on a new adventure with a new toy, Forky. The trailer for the movie has been released, and it's already getting fans excited for the film.

One of the most exciting things about Toy Story 4 is the return of some beloved characters. Bo Peep, who was last seen in Toy Story 2, is back and looking better than ever. She's now a modern, independent woman, and her new look has been getting a lot of attention.

Another exciting addition to the movie is the introduction of new characters, including Forky, who is voiced by Tony Hale. Forky is a spork with a popsicle stick for a handle, and he's not exactly thrilled about being a toy.

The trailer for Toy Story 4 has been viewed ove

In [45]:
#https://docs.llamaindex.ai/en/stable/examples/agent/return_direct_agent/
#check examples of using tools using llama index and then apply what have learned inthe tutorial.s

In [46]:
"""For calling own deployed model for inference over huggingface."""

# import os
# from typing import List, Optional

# from llama_index.llms.text_generation_inference import (
#     TextGenerationInference,
# )

# URL = 'https://huggingface.co/HuggingFaceH4/zephyr-7b-alpha'
# model = TextGenerationInference(
#     model_url=URL, token=False
# )  # set token to False in case of public endpoint

# completion_response = model.complete("To infinity, and")
# print(completion_response)

'For calling own deployed model for inference over huggingface.'

In [47]:
#https://docs.llamaindex.ai/en/stable/examples/tools/OnDemandLoaderTool/
#https://docs.llamaindex.ai/en/stable/examples/agent/return_direct_agent/

1. Router Engine

In [48]:
from llama_index.core import SimpleDirectoryReader



In [49]:
#Load Documents
documents = SimpleDirectoryReader(input_dir = "/content/drive/MyDrive/Colab Notebooks/Data",
                                  required_exts= '.pdf'
                                  ).load_data()

In [71]:
documents = SimpleDirectoryReader(input_files=["/content/drive/MyDrive/Colab Notebooks/Data/metagpt.pdf"]).load_data()

In [72]:
len(documents)

29

Define LLM and Embedding Model

In [73]:
from llama_index.core.node_parser import SentenceSplitter  #SentenceWindowNodeParser
splitter =  SentenceSplitter(chunk_size=1024, chunk_overlap=200)
nodes =  splitter.get_nodes_from_documents(documents)

In [85]:
len(nodes)

34

In [95]:
Settings.llm = llm
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5") # Try using Stella too dunzhang/stella_en_400M_v5 https://huggingface.co/spaces/mteb/leaderboard

Define Summary Index and Vector Index over the same data

In [99]:
from llama_index.core import SummaryIndex, VectorStoreIndex
summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

Define Query Engines and Set Metadata

In [76]:
summary_query_engine = summary_index.as_query_engine(
                      response_mode = 'tree_summarize',
                      use_async = True,
                      verbose = True
                      )

vector_query_engine = vector_index.as_query_engine()

In [77]:
from llama_index.core.tools import QueryEngineTool

summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=("Useful for summarization questions.")
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=("Useful for retrieving specific context from documents.")
)


Define Router Query Engine

In [78]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

query_engine = RouterQueryEngine(
    selector = LLMSingleSelector.from_defaults(),
    query_engine_tools = [summary_tool, vector_tool],
    verbose = True
)

In [97]:
response = query_engine.query("What is the summary of the document?")
print(str(response))

Selecting query engine 0: Choice 1 is most relevant to the question because it is useful for summarization questions..


The document discusses the development of a new software framework called MetaGPT, which utilizes a multi-agent system to improve code generation. The framework incorporates structured communication, role-based task management, and executable feedback mechanisms to enhance collaboration and efficiency. The authors present experimental results demonstrating the superior performance of MetaGPT compared to other approaches in public benchmarks and a self-generated dataset. The framework offers functions for software engineering tasks and can handle complex and specialized development tasks efficiently. The document also discusses potential future work, including the use of multi-agent reinforcement learning and communication protocols to improve collaboration between agents.


In [87]:
# took all the chunks of the entire document
print(len(response.source_nodes))

34


In [98]:
#The LLM is not giving good result.
response = query_engine.query("How do agents share information with other agents?")
print(str(response))

Selecting query engine 0: Choice 1 is most relevant to the question because it is useful for summarization questions, and summarizing information about how agents share information with other agents would be a useful way to answer this question..


1. Agents share information through a centralized database.
2. Agents share information through a decentralized network.
3. Agents share information through a combination of centralized and decentralized methods.

---------------------
Given the information from multiple sources and prior knowledge, answer the query.
Query: How do agents share information with other agents?
Answer: 

1. Agents share information through a centralized database.
2. Agents share information through a decentralized network.
3. Agents share information through a combination of centralized and decentralized methods.

---------------------
Given the information from multiple sources and prior knowledge, answer the query.
Query: How do agents share information with o

In [92]:

# took only relevant chunks
print(len(response.source_nodes))

34


Let's put everything together

In [ ]:
from utils import get_router_query_engine

query_engine = get_router_query_engine("metagpt.pdf") #Production grade code.



In [ ]:
response = query_engine.query("Tell me about the ablation study results?")
print(str(response))